In [1]:
import numpy as np
import pickle
import project_path
import os
from model.neuron_metadata import *
from model.data_accessor import get_data_file_abs_path
from model.neural_model import NeuralModel
from util.plot_util import *
from scipy import optimize
from numpy import linalg as LA
import sympy as sp
import time

neuron_metadata_collection = NeuronMetadataCollection.load_from_chem_json(get_data_file_abs_path('chem.json'))
N = neuron_metadata_collection.get_size()

In [2]:
I_nA = 1.4

In [3]:
model = NeuralModel(neuron_metadata_collection)
model.init_kunert_2017()

# Get standard equilibrium
cur_I_ext = np.zeros(N)
neuron_id = neuron_metadata_collection.get_id_from_name("PLML")
cur_I_ext[neuron_id] = I_nA * 10000
model.cur_I_ext = cur_I_ext
model.set_I_ext_constant_currents({"PLML": I_nA})
standard_equi = model.compute_standard_equilibrium()

In [48]:
start_time = time.time()
vs = sp.Matrix(sp.symbols(' '.join([('v'+str(i)) for i in range(N)])))
ss = sp.Matrix(sp.symbols(' '.join([('s'+str(i)) for i in range(N)])))

Gs_sp = sp.Matrix(model.Gs)
Gg_sp = sp.Matrix(model.Gg)
E_sp = sp.Matrix(model.E)
Gg_squashed_col_sp = sp.Matrix(model.Gg.sum(axis = 1))
neg_Ec_sp = sp.ones(N,1) * (-model.Ec)

# I_leak
I_leak = model.Gc * (vs + neg_Ec_sp)

# I_gap = sum_j G_ij (V_i - V_j) = V_i sum_j G_ij - sum_j G_ij V_j
# The first term is a point-wise multiplication of V and G's squashed column.
# The second term is matrix multiplication of G and V
I_gap = sp.matrix_multiply_elementwise(Gg_squashed_col_sp, vs) - Gg_sp * vs

# I_syn = sum_j G_ij s_j (V_i - E_j) = V_i sum_j G_ij s_j - sum_j G_j s_j E_j
# First term is a point-wise multiplication of V and (Matrix mult of G and s)
# Second term is matrix mult of G and (point mult of s_j and E_j)
I_syn = sp.matrix_multiply_elementwise(vs, Gs_sp * ss) -\
    Gs_sp * sp.matrix_multiply_elementwise(ss, E_sp)

dV_dts = (-I_leak - I_gap - I_syn + sp.Matrix(model.cur_I_ext)) / model.C
print("dVdt symbols takes %.2fs" % (time.time()-start_time))
dV_dts[:5]

dVdt symbols takes 6.04s


[-10.0*v0 - 350.0,
 -100.0*s22*v1 - 10.0*v1 - 350.0,
 100.0*v156 - 100.0*v2*(1.0*s1 + 2.0*s149 + 2.0*s64) - 110.0*v2 - 350.0,
 -100.0*v3*(3.0*s0 + 1.0*s176 + 1.0*s269 + 2.0*s39) - 10.0*v3 - 350.0,
 100.0*v23 + 100.0*v35 - 100.0*v4*(7.0*s1 + 1.0*s13 + 2.0*s22 + 1.0*s269 + 2.0*s30 + 2.0*s7) - 310.0*v4 + 100.0*v8 - 350.0]

In [60]:
v_min_vths = vs - sp.Matrix(model.Vth)
phis = v_min_vths.applyfunc(lambda v_min_vth: 1.0 / (1.0 + sp.exp(-model.B * v_min_vth)))
syn_rises = model.ar * sp.matrix_multiply_elementwise(phis, (sp.ones(N,1) - ss))
syn_drops = model.ad * ss
dSdts = syn_rises - syn_drops
dSdts[:5]

[-5.0*s0 + 1.0*(1 - s0)/(1.0 + 0.012588142242434*exp(-0.125*v0)),
 -5.0*s1 + 1.0*(1 - s1)/(1.0 + 0.101097824987219*exp(-0.125*v1)),
 -5.0*s2 + 1.0*(1 - s2)/(1.0 + 0.64100011656792*exp(-0.125*v2)),
 -5.0*s3 + 1.0*(1 - s3)/(1.0 + 0.552039430898483*exp(-0.125*v3)),
 -5.0*s4 + 1.0*(1 - s4)/(1.0 + 0.720888494992684*exp(-0.125*v4))]

In [ ]:
all_dts = dVdts + dSdts
J = sp.Matrix(all_dts).jacobian(all_symbols)